# Build and train a fast.ai model with Amazon SageMaker Notebook instance

## Background

This notebook shows how to build and train a fast.ai based Deep Learning model using an Amazon SageMaker notebook instance, save the model and upload the model artefacts to S3.

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
bucket='sagemaker-mcclean-eu-west-1'          # customize to the name of your S3 bucket
model_file_name = 'lhr-summit-demo'           # customize to the name of your model
PATH='data/dogscats/'                         # customize to the relative location of your data folder
key='models/'+model_file_name+'/model.tar.gz' # prefix of the S3 bucket of the model file

In [4]:
import boto3
import re
import os

## Train fastai model locally
Now we will train the fastai CNN model to distinguish between cats and dogs based on lesson 1 of the fast.ai course.

In [5]:
import torch
from fastai.imports import *

In [6]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [7]:
sz=224

In [8]:
torch.cuda.is_available()

True

In [9]:
torch.backends.cudnn.enabled

True

In [10]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/ec2-user/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:01<00:00, 60220723.64it/s]


100%|██████████| 32/32 [00:05<00:00,  5.89it/s]


epoch      trn_loss   val_loss   accuracy                     
    0      0.05033    0.031138   0.99      
    1      0.042945   0.025746   0.991                        



[array([0.02575]), 0.991]

Now we can save our model so it can be used later by SageMaker using it as a model hosting service.

In [11]:
learn.save(model_file_name)

## Upload fastai model to S3
Now that we have trained a model and saved locally we can upload it plus some extra files to S3.

In [12]:
def get_relative_path(filename):
    s1 = os.path.split(filename)
    p = os.path.split(s1[0])[1]
    return os.path.join(p, s1[1])

In [13]:
def create_dummy_data(src_path, dest_root, sub_dir, num_items=2):
    if not os.path.isdir(dest_root): os.mkdir(dest_root)
    dst_path = os.path.join(dest_root, sub_dir)
    classes = os.listdir(src_path)
    for d in classes:
        if d.startswith('.'): continue
        if not os.path.isdir(dst_path): os.mkdir(dst_path)
        if not os.path.isdir(os.path.join(dst_path, d)): os.mkdir(os.path.join(dst_path, d))
        fnames = glob('{}/{}/*.jpg'.format(src_path, d))
        for i in range(num_items):
            shutil.copyfile(fnames[i], os.path.join(dst_path, get_relative_path(fnames[i])))

In [14]:
create_dummy_data(PATH + "train", PATH + "models/data", "train")
create_dummy_data(PATH + "valid", PATH + "models/data", "valid")

Now we will create a tarfile with the saved model and some dummy images from the train and validation data sets.

In [15]:
!tar czvf data/dogscats/model.tar.gz -C data/dogscats/models/ .

./
./data/
./data/valid/
./data/valid/dogs/
./data/valid/dogs/dog.7496.jpg
./data/valid/dogs/dog.6748.jpg
./data/valid/cats/
./data/valid/cats/cat.5352.jpg
./data/valid/cats/cat.9144.jpg
./data/train/
./data/train/dogs/
./data/train/dogs/dog.7316.jpg
./data/train/dogs/dog.2429.jpg
./data/train/cats/
./data/train/cats/cat.7238.jpg
./data/train/cats/cat.10578.jpg
./lhr-summit-demo.h5


Now we will upload the zipped model and example images to S3.

In [16]:
boto3.client('s3').upload_file(PATH+"model.tar.gz", bucket, key)
print(f"Uploaded model artefacts to s3://{bucket}/{key}")

## Conclusion

You have seen in this example notebook how to train an example fast.ai model on a SageMaker notebook, save the model and upload it to S3.